In [6]:
#               TEST cluster to bring up the cluster session
a=2

An error was encountered:
Session 8 did not reach idle status in time. Current status is busy.


In [ ]:
#https://spark.apache.org/docs/2.1.0/ml-classification-regression.html#decision-tree-classifier

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [3]:
from pyspark.ml.feature import OneHotEncoder

In [5]:
###              Import all the datasets from Azure Storage Container

customerdetails = spark.read.csv("wasb://hackathonspark-2018-06-20t07-09-42-165z@hackathonspark.blob.core.windows.net/puneeth_delete/customerdetails.csv", header=True, inferSchema=True)
customer_cdr = spark.read.csv('wasb://hackathonspark-2018-06-20t07-09-42-165z@hackathonspark.blob.core.windows.net/puneeth_delete/customer_cdr.csv',header=True,inferSchema=True)
plan_details = spark.read.csv('wasb://hackathonspark-2018-06-20t07-09-42-165z@hackathonspark.blob.core.windows.net/puneeth_delete/plan_details.csv',header=True,inferSchema=True)

An error was encountered:
Session 8 did not reach idle status in time. Current status is busy.


In [ ]:
##                         JOIN THE DATASETS INTO A SINGLE DATAFRAME
data = customer_details.join(customer_cdr,"customer_unique_id").join(plan_details,"customer_unique_id").join(signal_details, "customer_unique_id")

In [ ]:
#                              DATA PREPRCOCESSING 

In [ ]:
#Replace NAN with 0 calls.
data['customer_service_calls'].fillna(0, inplace=True)
data['international_plan'].fillna('no', inplace=True)
data['voice_mail_plan'].fillna('no', inplace=True)

In [5]:
cols = data.columns

In [6]:
#                               count rows in DATA
data.count()

3333

In [7]:
#                                check column data types 
data.dtypes

[('state', 'string'), ('account_length', 'int'), ('area_code', 'int'), ('phone_number', 'string'), ('international_plan', 'string'), ('voice_mail_plan', 'string'), ('number_vmail_messages', 'int'), ('total_day_minutes', 'double'), ('total_day_calls', 'int'), ('total_day_charge', 'double'), ('total_eve_minutes', 'double'), ('total_eve_calls', 'int'), ('total_eve_charge', 'double'), ('total_night_minutes', 'double'), ('total_night_calls', 'int'), ('total_night_charge', 'double'), ('total_intl_minutes', 'double'), ('total_intl_calls', 'int'), ('total_intl_charge', 'double'), ('customer_service_calls', 'int'), ('churn', 'string')]

In [8]:
categoricalColumns = ['state','phone_number','international_plan','voice_mail_plan']

In [9]:
stages = [] # stages in our Pipeline
stringIndexer = StringIndexer(inputCol='state', outputCol='stateIndex')
encoder = OneHotEncoder(inputCol='stateIndex', outputCol='stateclassVec')
stages += [stringIndexer, encoder]
stringIndexer = StringIndexer(inputCol='phone_number', outputCol='phone_numberIndex')
encoder = OneHotEncoder(inputCol='phone_numberIndex', outputCol='phone_numberclassVec')
stages += [stringIndexer, encoder]
stringIndexer = StringIndexer(inputCol='international_plan', outputCol='international_planIndex')
encoder = OneHotEncoder(inputCol='international_planIndex', outputCol='international_planclassVec')
stages += [stringIndexer, encoder]
stringIndexer = StringIndexer(inputCol='voice_mail_plan', outputCol='voice_mail_planIndex')
encoder = OneHotEncoder(inputCol='voice_mail_planIndex', outputCol='voice_mail_planclassVec')
stages += [stringIndexer, encoder]

In [10]:
label_stringIdx = StringIndexer(inputCol="churn", outputCol="label")
stages += [label_stringIdx]

In [11]:
numericCols = ['account_length','area_code','number_vmail_messages','total_day_minutes','total_day_calls','total_day_charge','total_eve_minutes','total_eve_calls','total_eve_charge','total_night_minutes','total_night_calls','total_night_charge','total_intl_minutes','total_intl_calls','total_intl_charge','customer_service_calls']     

In [12]:
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [13]:
pipeline = Pipeline(stages=stages)

In [14]:
pipelineModel = pipeline.fit(data)

In [15]:
data = pipelineModel.transform(data)

In [16]:
data.columns#

['state', 'account_length', 'area_code', 'phone_number', 'international_plan', 'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes', 'total_day_calls', 'total_day_charge', 'total_eve_minutes', 'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 'total_night_calls', 'total_night_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'customer_service_calls', 'churn', 'stateIndex', 'stateclassVec', 'phone_numberIndex', 'phone_numberclassVec', 'international_planIndex', 'international_planclassVec', 'voice_mail_planIndex', 'voice_mail_planclassVec', 'label', 'features']

In [17]:
selectedcols = ["label", "features"] + cols
data = data.select(selectedcols)
#display(data)

In [18]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=100)

In [19]:
############################################## LOGISTIC REGRESSION

In [20]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [21]:
predictions = lrModel.transform(testData)

In [ ]:
predictions.printSchema()

In [ ]:
testData.count()


In [ ]:
130+849+10+2

In [ ]:
testData.filter(testData.churn=='Yes').count()

In [ ]:
testData.filter(testData.churn=='No').count()

In [22]:
# df.stat.crosstab("name", "item").show()
# When Churn= 'No' label= 0.0, most frequently occuring item is assigned 0.0
# When Churn= 'Yes' label= 1.0

# Prediction = 0.0  churn = No
# Prediction = 1.0  churn = Yes
predictions.stat.crosstab("label","prediction").show()

+----------------+---+---+
|label_prediction|0.0|1.0|
+----------------+---+---+
|             1.0|130| 10|
|             0.0|849|  2|
+----------------+---+---+

In [ ]:
selected = predictions.select("label", "prediction", "state")
#display(selected)

In [ ]:
type(predictions)

In [ ]:
predictions.count()

In [ ]:
predictions.columns

In [ ]:
#predictions.write.csv('wasb://hackpyspark-2018-06-21t05-44-31-166z@lyoltv4o2zxni.blob.core.windows.net/puneeth_delete/trial_written.csv')

In [ ]:
predictions.dtypes

In [ ]:
records = predictions.filter(~predictions.churn.isin('Yes'))
#records = predictions.filter(~predictions.churn.isin('No'))

In [ ]:
records.take(2)

In [23]:
#to evaluate the model using area under the ROC curve
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.7757680040288712

In [ ]:
evaluator.getMetricName()

In [24]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [25]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [26]:
# Use test set to measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [27]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)


0.8230653013261753

In [28]:
# df.stat.crosstab("name", "item").show()
# When Churn= 'No' label= 0.0, most frequently occuring item is assigned 0.0
# When Churn= 'Yes' label= 1.0

# Prediction = 0.0  churn = No
# Prediction = 1.0  churn = Yes
predictions.stat.crosstab("label","prediction").show()

+----------------+---+---+
|label_prediction|0.0|1.0|
+----------------+---+---+
|             1.0|139|  1|
|             0.0|850|  1|
+----------------+---+---+

In [ ]:
############################################## DECISION TREES

In [29]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(trainingData)

In [ ]:
print("numNodes = ", dtModel.numNodes)
print("depth = ", dtModel.depth)

In [30]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)


In [ ]:
predictions.printSchema()

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.20144367970454927

In [32]:
# df.stat.crosstab("name", "item").show()
# When Churn= 'No' label= 0.0, most frequently occuring item is assigned 0.0
# When Churn= 'Yes' label= 1.0

# Prediction = 0.0  churn = No
# Prediction = 1.0  churn = Yes
predictions.stat.crosstab("label","prediction").show()

+----------------+---+---+
|label_prediction|0.0|1.0|
+----------------+---+---+
|             1.0| 88| 52|
|             0.0|844|  7|
+----------------+---+---+

In [33]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1, 2, 6, 10])
             .addGrid(dt.maxBins, [20, 40, 80])
             .build())

In [34]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# Takes ~5 minutes

In [ ]:
print("numNodes = ", cvModel.bestModel.numNodes)
print("depth = ", cvModel.bestModel.depth)

In [35]:
# Use test set to measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [36]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.4834102736276649

In [37]:
# df.stat.crosstab("name", "item").show()
# When Churn= 'No' label= 0.0, most frequently occuring item is assigned 0.0
# When Churn= 'Yes' label= 1.0

# Prediction = 0.0  churn = No
# Prediction = 1.0  churn = Yes
predictions.stat.crosstab("label","prediction").show()

+----------------+---+---+
|label_prediction|0.0|1.0|
+----------------+---+---+
|             1.0| 39|101|
|             0.0|841| 10|
+----------------+---+---+

In [ ]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction","state")
display(selected)

In [ ]:
#####################################################  RANDOM FORREST

In [38]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [39]:
# Make predictions on test data using the Transformer.transform() method.
predictions = rfModel.transform(testData)

In [ ]:
predictions.printSchema()

In [ ]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "state")
display(selected)


In [40]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.8443218062783273

In [41]:
# df.stat.crosstab("name", "item").show()
# When Churn= 'No' label= 0.0, most frequently occuring item is assigned 0.0
# When Churn= 'Yes' label= 1.0

# Prediction = 0.0  churn = No
# Prediction = 1.0  churn = Yes
predictions.stat.crosstab("label","prediction").show()

+----------------+---+
|label_prediction|0.0|
+----------------+---+
|             1.0|140|
|             0.0|851|
+----------------+---+

In [42]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [43]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)

In [44]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [45]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.8173115662246083

In [ ]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "state")
display(selected)


In [46]:
# df.stat.crosstab("name", "item").show()
# When Churn= 'No' label= 0.0, most frequently occuring item is assigned 0.0
# When Churn= 'Yes' label= 1.0

# Prediction = 0.0  churn = No
# Prediction = 1.0  churn = Yes
predictions.stat.crosstab("label","prediction").show()

+----------------+---+
|label_prediction|0.0|
+----------------+---+
|             1.0|140|
|             0.0|851|
+----------------+---+

In [49]:
#distribution of test and traindata
trainingData.stat.crosstab("churn","churn").show()

+-----------+----+---+
|churn_churn|  No|Yes|
+-----------+----+---+
|         No|1999|  0|
|        Yes|   0|343|
+-----------+----+---+

In [50]:
1999+343

2342

In [51]:
testData.stat.crosstab("churn","churn").show()

+-----------+---+---+
|churn_churn| No|Yes|
+-----------+---+---+
|         No|851|  0|
|        Yes|  0|140|
+-----------+---+---+

In [52]:
851+140

991

In [ ]:
#####################################################################################################
##https://docs.databricks.com/spark/latest/mllib/binary-classification-mllib-pipelines.html
##    ##        ##        #####  ##   ##     ## ########################################################################################
##    ##       ###      ###      ##  ##     #### #######################################################################################
########      ## ##    ###       ## ##     ###### ###############################################################################
########     #######   ###       ####     ######## ###########################################################################
##    ##    ##     ##    ###     ## ##   ##      ## ###########################################################################
##    ##   ##       ##    #####  ##  ## ##        ## #####################################################################

#####################################################################################################
#####################################################################################################
#####################################################################################################
#####################################################################################################     START